In [ ]:
# Library imports
import os
import sys
import openpyxl


In [ ]:
# Variable setup
tempFile = "temp.xlsx"

In [ ]:
# Create new workbook then save
wb = openpyxl.Workbook() # create new workbook
wb.save(filename=tempFile)

In [ ]:
# Setup default content of the workbook for below tasks
# No need to understand these now, you will understand when finish reading the rest.
wb = openpyxl.load_workbook(filename=tempFile)
# Create content in the default sheet with monthly sales data
if 'sales' in wb.sheetnames:
	ws = wb['sales']
else:
	ws = wb.create_sheet('sales')
	ws["A1"].value = "Month"
	ws["B1"].value = 'Sales'

	for i in range(1,13):
		row = i + 1
		cellARef = 'A' + str(row)
		cellBRef = 'B' + str(row)
		ws[cellARef].value = i
		ws[cellBRef].value = i * 100

# Create a new product sheet then put in default data
if 'products' in wb.sheetnames:
	ws = wb['products']
else:
	ws = wb.create_sheet('products')
	ws["A1"].value = "Product Id"
	ws["B1"].value = 'Title'
	ws["C1"].value = "Parent"
	ws["D1"].value = 'Category'

	for row in range (2, 4):
		for col in range(1, 5):
			ws.cell(row=row, column= col, value=row + col)

# Create a new review sheet then put in default data
if 'reviews' in wb.sheetnames:
	ws = wb['reviews']
else:
	ws = wb.create_sheet('reviews')
	ws["A1"].value = "Review Id"
	ws["B1"].value = 'Customer ID'
	ws["C1"].value = "Stars"
	ws["D1"].value = 'Date'

	for row in range (2, 4):
		for col in range(1, 5):
			ws.cell(row=row, column= col, value=row * col)

wb.save(filename=tempFile)

In [ ]:
# Load Workbook
wb = openpyxl.load_workbook(filename=tempFile, read_only=True) # can set addition options for load workbook
print(wb.sheetnames)
ws = wb.active
# ws = wb['ws']
print(ws)
print(ws.title)

print(f"Total number of rows: {ws.max_row}. Total number of columns: {ws.max_column}")

wb.close() # here use close to close the file. below will use save, save will close file as well

In [ ]:
# Create, duplicate and remove Work sheets
wb = openpyxl.load_workbook(filename=tempFile)
wb.create_sheet("new sheet")
print(wb.sheetnames)
ws = wb['new sheet']
print(ws)
print(ws.title) # print title

ws.title = 'new title now' # change sheet name
print(ws.title) # print title

wb.copy_worksheet(wb['new title now'])

wb.remove(wb["new title now"]) # or del wb['new ws']

wb.save(filename=tempFile)

In [ ]:
# Set and retrieve cell data
wb = openpyxl.load_workbook(filename=tempFile)
ws = wb.create_sheet('SetRetrieveData')

# Use Excel column and row notation
ws['A1'].value = "Hello World" # Set A1 cell value

print(ws['A1'])
print(ws['A1'].value) # To get the actual value, need .value

# Use row and column number by using cell object, This is easier to be used in loop
cell = ws.cell(row=10, column=6)
print(cell)
cell.value = "I'm the new value"
print(cell.value)
print(ws['F10'].value)

ws.cell(row=2, column=2, value='test') # a new way to set cell value
print(ws['B2'].value)
print(ws.cell(row=2, column=2).value) # use cell to retrieve value.

ws['B1'].value = 5
ws['B2'].value = 6
ws['B3'].value = '=sum(B1:B2)' # Put formulas into the cell
print(ws['B3'].value) # will print formulas, but when open the workbook, it will be value

wb.remove(wb['SetRetrieveData'])

wb.save(filename=tempFile)

In [ ]:
# Appending / delete rows / create columns
# Append new row - use ws.append, cannot do this for columns
wb = openpyxl.load_workbook(filename=tempFile)
ws = wb['Sheet']

new_row = (1,2,3,'the 4 column', 5)
ws.append(new_row)
print('New row appended')

# create new column
ws['F1'].value = 1
ws.cell(row=2, column=6,value=2)
ws['F3'].value = 'three'
print('Added value into F column')

ws.delete_rows(ws.max_row, 1) # delete last row
print('Deleted the last row created')

wb.save(tempFile)

In [ ]:
# Slicing and iterating through data
wb = openpyxl.load_workbook(filename=tempFile)
ws = wb['Sheet']

data = ws["A:B"] # slice a 2D area, data is tuple, not list
# data = ws["A"] # a slice of A column
# data = ws[5] # a slice of 5th row
# data = ws[5:6] # a slice of row 5 and 6
print(data)
print("1---")

for row in ws.iter_rows(min_row=1, max_row=13, min_col=1, max_col=2, values_only=True): 
	# Above the value_only print the value instead of cell reference such as <Cell 'Sheet'.A1>
	print(row)
print("2---")

for col in ws.iter_cols(min_row=1, max_row=13, min_col=1, max_col=2):
	for e in col: # Can iterate through column element using normal loop, this also apply to row above.
		print(e)
	print(col)
print("3---")

# If want to go through the whole worksheet, can use ws.rows or ws.columns
for row in ws.rows:
	for e in row:
		print(e.value) # This print cell values in each row.
	print(row) # This print cell reference

In [ ]:
# Manipulate Data
import json

wb = openpyxl.load_workbook(filename=tempFile, read_only=True)
ws = wb['sales']

sales = {} # declare dictionary

# Using the values_only because I only want to return the cells' values
for row in ws.iter_rows(min_row=2, min_col=1, values_only=True):
    month = row[0] # Although in openpyxl the index starts at 1, python list, dictionary starts at 0
    sale = {
        "month": row[0],
        "sales": row[1]
    }
    sales[month] = sale

wb.close()

# print in json format
json_string = json.dumps(sales)
print(json_string) 


# only to pretty print json, more human readable. no use in data processing
parsed = json.loads(json_string) # json.loads() take a json string. 
print(json.dumps(parsed, indent=2))

In [ ]:
# Convert data into Python Classes

# First define class
import dataclasses

@dataclasses.dataclass
class Product:
	id: str
	title: str
	parent: str
	category: str

@dataclasses.dataclass
class Review:
	id: str
	customer_id: str
	stars: int
	date: str

# then define the mapping of column to the excel file start index from 0 - this step is not mandatory but recommended
CATEGORY = 3
PRODUCT_ID = 0
TITLE = 1
PARENT = 2

STARS = 2
REVIEW_ID = 0
CUSTOMER = 1
DATE = 3

# Create Lists for products and reviews type
products = []
reviews = []

# read in date from products sheet, create objects and then add to list
wb = openpyxl.load_workbook(filename=tempFile)
ws = wb['products']
for row in ws.iter_rows(min_row=2, min_col=1, values_only=True):
	product = Product(id=row[PRODUCT_ID], title=row[TITLE], parent = row[PARENT], category = row[CATEGORY])
	products.append(product)

print(products) # Print while list
print(json_string)

print("1---")

# read in date from review sheet, create objects and then add to list
ws = wb['reviews']
for row in ws.iter_rows(min_row=2, min_col=1, values_only=True):
	review = Review(id=row[REVIEW_ID], customer_id=row[CUSTOMER], stars = row[STARS], date = row[DATE])
	reviews.append(review)
print(reviews[0])
print("2---")

In [ ]:
# Convert Python classes to Excel 

In [ ]:
# Clean up

if os.path.isfile(tempFile):
	os.remove(tempFile)
	# os.rmdir() # Remove an empty directory
	print(f'{tempFile} has been removed')
else:
	print(f"{tempFile} not found.")
print("Clean up completed.")